## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-22-57-28 +0000,nyt,Venezuelan Lawmakers Approve Sweeping Overhaul...,https://www.nytimes.com/2026/01/29/world/ameri...
1,2026-01-29-22-55-09 +0000,wapo,Dozens of Saks Off 5th stores to close as Saks...,https://www.washingtonpost.com/business/2026/0...
2,2026-01-29-22-54-37 +0000,bbc,Scotland's first deputy first minister Lord Ji...,https://www.bbc.com/news/articles/cx2l3dy0llro...
3,2026-01-29-22-53-31 +0000,nyt,Trump Says Putin Agreed to a Weeklong Pause in...,https://www.nytimes.com/2026/01/29/us/politics...
4,2026-01-29-22-49-00 +0000,wsj,"Sandisk Profit, Revenue Jump on AI Demand",https://www.wsj.com/business/earnings/sandisk-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,51
375,new,23
109,ice,22
407,minnesota,17
592,he,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
143,2026-01-29-18-17-00 +0000,wsj,President Trump said the White House is moving...,https://www.wsj.com/politics/policy/white-hous...,133
89,2026-01-29-20-34-23 +0000,nypost,Trump prevents ‘boring’ cabinet meeting so he ...,https://nypost.com/2026/01/29/us-news/trump-sp...,117
193,2026-01-29-15-47-21 +0000,nyt,Trump Moves to Sell Former Home of Capital’s T...,https://www.nytimes.com/2026/01/28/us/politics...,116
325,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,114
350,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
143,133,2026-01-29-18-17-00 +0000,wsj,President Trump said the White House is moving...,https://www.wsj.com/politics/policy/white-hous...
236,93,2026-01-29-13-31-00 +0000,wsj,Democratic Sen. Amy Klobuchar of Minnesota ann...,https://www.wsj.com/politics/elections/sen-amy...
344,87,2026-01-29-03-15-00 +0000,wsj,Google co-founder Sergey Brin has contributed ...,https://www.wsj.com/politics/policy/sergey-bri...
256,82,2026-01-29-12-00-01 +0000,startribune,AI image of Alex Pretti’s killing is the lates...,https://www.startribune.com/ai-image-of-alex-p...
325,54,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
68,51,2026-01-29-21-11-32 +0000,nypost,Rudy Giuliani trashes Mamdani’s NYC homeless p...,https://nypost.com/2026/01/29/us-news/rudy-giu...
350,47,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...
99,43,2026-01-29-20-05-00 +0000,wsj,Tulsi Gabbard has been investigating results o...,https://www.wsj.com/politics/elections/spy-chi...
36,41,2026-01-29-21-57-12 +0000,nypost,Saks Off 5th to close most stores following ba...,https://nypost.com/2026/01/29/business/saks-of...
206,37,2026-01-29-15-19-00 +0000,wsj,"After pandemic-era hiring binges, big companie...",https://www.wsj.com/economy/jobs/u-s-companies...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
